In [17]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
import random
from sklearn.preprocessing import OneHotEncoder 
from scipy.sparse import coo_matrix, hstack,vstack
import time
from sklearn.utils import shuffle

In [18]:
df=pd.read_csv("NetflixProcessedDataFile.csv",index_col=0).drop_duplicates()
print(df.iloc[::5000000, :])#START:STOP:STEP (5000000)

C:\Users\Veronika_Revyakina\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


           User_Id  Rating  Movie_Id
1          1488844     3.0         1
5000996     501954     2.0       996
10001962    404654     5.0      1962
15002876    886608     2.0      2876
20003825   1193835     2.0      3825
25004661   1899206     3.0      4661
30005496    154804     4.0      5496
35006274   2078749     5.0      6274
40007057    450763     5.0      7057
45007991    102092     3.0      7991
50009023    220298     5.0      9023
55010042    550530     5.0     10042
60011038    222570     3.0     11038
65011875   1273080     5.0     11875
70012676   2026970     5.0     12676
75013582    506044     4.0     13582
80014453    353605     2.0     14453
85015116    664606     3.0     15116
90016008   2213715     3.0     16008
95016879   1589401     5.0     16879
100017627  2314006     4.0     17627


In [19]:
# Get one hot encoding of column User_id
#one_hot_users = pd.get_dummies(df1['User_Id'])
#one_hot_users
# Drop column User_Id as it is now encoded
#df1 = df1.drop('User_Id',axis = 1)
# Join the encoded df
#df1 = df1.join(one_hot_users)
#df1  

#This approach with dummies wouldnt work, because dataset is too large

In [20]:
# importing one hot encoder from sklearn 
encoder = OneHotEncoder(categories='auto') 
#X-train matrix for encoder to learn categories
#enc.fit(X)  #process of encoder learning categories
one_hot_user_matrix = encoder.fit_transform(np.asarray(df['User_Id']).reshape(-1,1)) 
one_hot_movie_matrix = encoder.fit_transform(np.asarray(df['Movie_Id']).reshape(-1,1)) 
print(one_hot_movie_matrix.shape) #amount of rows=amount of ratings , amount of columns=amount of unique user_ids

(100480507, 17770)


In [21]:
X=hstack([one_hot_user_matrix,one_hot_movie_matrix]) #24053764x475257 matrix joined user_id and movie_id 2 groups
#475257=amount of users+amount of films
#24053764=amount of non null ratings
#create ratings vector 
ratings=np.asarray(df['Rating']).reshape(-1,1)
#ratings=df['Rating']
print(X.shape)
print(ratings.shape)
X,ratings=shuffle(X,ratings)

(100480507, 497959)
(100480507, 1)


In [22]:
def svd_predict(X,omega,omega0,V,sum_const):
    prod=X.dot(omega)#(24053764,1)
    linear_part=omega0+prod
    #V_part=np.square(sum_const)-X.power(2).dot(np.square(V)) # (24053764,3)
    V_part=(sum_const**2)-X.power(2).dot((V)**2) 
    sqr_part=(0.5*np.sum(V_part,axis=1)).reshape(-1,1) # (24053764,1)
    return linear_part+sqr_part # (24053764,1)
    
def root_mean_square_error(ratings_predict,ratings_actual):
    #ratings_predict=svd_predict(X,omega,omega0,V)
    #summands=np.power(ratings_actual-ratings_predict,2)
    #return np.sqrt(mean_squared_error(ratings_actual, ratings_predict))
    mse=np.sum(np.power(ratings_actual-ratings_predict,2))/len(ratings_predict)
    return np.sqrt(mse)
   
def r_squared(ratings_predict,ratings_actual):
    #ratings_predict=svd_predict(X,omega,omega0,V)
    nom=np.sum(np.power(ratings_actual-ratings_predict,2))
    denom=np.sum(np.power(ratings_actual-np.mean(ratings_actual,axis=0),2))
    return 1-nom/denom

In [24]:
def mini_batch_gradient_descent(X_train,Y_train,K,learning_rate,num_epoches,batch_size):
    #omega=np.random.sample((X_train.shape[1],1)) 
    #omega=(np.array([0.5]*(X_train.shape[1])).reshape(X_train.shape[1],1))
    omega=np.full((X_train.shape[1], 1), 0.5)
    omega0=0.5
    #V=np.random.sample((X_train.shape[1],K))
    #V=(np.array([0.5]*(X_train.shape[1]*K)).reshape(X_train.shape[1],K))
    V=np.full((X_train.shape[1], K), 0.5)
    r2_tr=[]
    rmse_tr=[]
   
    it=0
    for it in range(num_epoches): 
        #data = sparse.hstack((X_train, Y_train)).tocsr()
        #np.random.shuffle(data) #shuffle at each epoch (not every batch creation)
        #shuffle_data(data)
        X_train,Y_train=shuffle(X_train,Y_train)
        n_minibatches = X_train.shape[0] // batch_size 
        i = 0
        for i in range(n_minibatches + 1):
            X_mini=X_train.tocsr()[i * batch_size:(i + 1)*batch_size, :] 
            y_mini=Y_train[i * batch_size:(i + 1)*batch_size,:]
            #print(np.shape(X_mini))   
            #print(np.shape(y_mini)) 
            N=X_mini.shape[0] # 24053764
            #PARAMETERS
            sum_const=X_mini.dot(V)
            ratings_predict=svd_predict(X_mini,omega,omega0,V,sum_const)
            error=y_mini-ratings_predict #(24053764,1)
            omega=omega+learning_rate*2*((X_mini.T).dot(error))/N #(475257,1)
            omega0=omega0+learning_rate*2*np.sum(error)/N #(1,1)
            V = V + learning_rate*2*((X_mini.T.dot(np.multiply(error,sum_const)))-np.multiply(V, X_mini.T.dot(error)))/N

            ratings_predict=svd_predict(X_mini,omega,omega0,V,sum_const)
            r2_tr.append(r_squared(ratings_predict, y_mini))
            rmse_tr.append(root_mean_square_error(ratings_predict, y_mini))
            #print(rmse_tr)
            #print(r2_tr)
    return omega,omega0,V

In [25]:
r2=[]
rmse=[]
omega=[]
omega0=[]
r2_train=[]
rmse_train=[]
cross_val_parts_amount=5
cross_val_part_size=X.shape[0]//cross_val_parts_amount
item=0
for item in range(cross_val_parts_amount):
    X_test=X.tocsr()[item*cross_val_part_size:(item+1)*cross_val_part_size,:]
    Y_test=ratings[item*cross_val_part_size:(item+1)*cross_val_part_size]
    X_train=vstack([X.tocsr()[0:(item)*cross_val_part_size,:], X.tocsr()[(item+1)*cross_val_part_size:X.shape[0],:]], 'csr') 
    #Y_train=np.delete(ratings,ratings[item*cross_val_part_size:(item+1)*cross_val_part_size])
    #Y_train=ratings.drop(ratings.index[item*cross_val_part_size:(item+1)*cross_val_part_size])
    Y_train=np.vstack((ratings[0:item*cross_val_part_size],ratings[(item+1)*cross_val_part_size:ratings.shape[0]]))
    #print(np.shape(X_train))   
    #print(np.shape(Y_train))   
    start = time.time()
    omega_item,omega0_item,V_item=mini_batch_gradient_descent(X_train,Y_train,K=3,learning_rate = 0.01,num_epoches=3,
                                                            batch_size=100000)
    stop = time.time()
    duration = stop-start
    print(duration)
    sum_const=X_test.dot(V_item)                     
    ratings_predict=svd_predict(X_test,omega_item,omega0_item,V_item,sum_const)
    r2.insert(item,r_squared(ratings_predict,Y_test))
    rmse.insert(item,root_mean_square_error(ratings_predict,Y_test))
    sum_const=X_train.dot(V_item)                     
    ratings_predict=svd_predict(X_train,omega_item,omega0_item,V_item,sum_const)                    
    r2_train.insert(item,r_squared(ratings_predict,Y_train))
    rmse_train.insert(item,root_mean_square_error(ratings_predict,Y_train))
    print(r2[item],rmse[item],r2_train[item],rmse_train[item])

469.857412815094
0.010662515055604116 1.0793783636805452 0.01067270100561446 1.0794216932390956
474.4721620082855
0.010659036708014491 1.0793890778481028 0.010674899707278529 1.0794182768248748
474.1438162326813
0.010674330676887833 1.0794801291164673 0.010664257958878487 1.0793992479912322
471.3788721561432
0.010671055953146436 1.0793984197454107 0.010667797170874072 1.0794181907338827
503.83174419403076
0.010664635786712728 1.0794309138930966 0.010672378610222744 1.079408431481405


In [26]:
# initialise data of lists. 
data = {'1':[rmse[0],r2[0],rmse_train[0],r2_train[0]], 
        '2':[rmse[1],r2[1],rmse_train[0],r2_train[1]],
        '3':[rmse[2],r2[2],rmse_train[2],r2_train[2]],
        '4':[rmse[3],r2[3],rmse_train[3],r2_train[3]],
        '5':[rmse[4],r2[4],rmse_train[4],r2_train[4]],
        'E':[np.mean(rmse),np.mean(r2),np.mean(rmse_train),np.mean(r2_train)],
        'SD':[np.std(rmse),np.std(r2),np.std(rmse_train),np.std(r2_train)]} 
 
# Creates pandas DataFrame. 
df_res = pd.DataFrame(data, index =['RMSE', 'R^2','RMSE train','R^2 train']) 

df_res

,1,2,3,4,5,E,SD
RMSE,1.079378,1.079389,1.079480,1.079398,1.079431,1.079415,0.000037
R^2,0.010663,0.010659,0.010674,0.010671,0.010665,0.010666,0.000006
RMSE train,1.079422,1.079422,1.079399,1.079418,1.079408,1.079413,0.000008
R^2 train,0.010673,0.010675,0.010664,0.010668,0.010672,0.010670,0.000004
